# Maximum Likelihood Estimation (MLE) and Maximum A Posteriori (MAP) estimation

## Warmup to MLE and MAP

We will introduce the ideas of MLE and MAP in a simple discrete case.

#### MLE

You are given a coin and you are told that it is weighted to come up heads with probability $\theta$.  You toss the coin $3$ times and obtain the sequence HHT.  Based only on this information, what is your best estimate for $\theta$?

One way to address this rigorously is to rephrase the question this way:  of all coins with weightings $\theta \in [0,1]$, which value of $\theta$ gives the **maximum likelihood** of producing the sequence HHT?  First make a guess about what is reasonable!

Note:  We use the term **likelihood** to refer to the probability of the observed data as a function of the parameters.  The likelihood is [not a probability density function](https://stats.stackexchange.com/a/31241).

The Likelihood of a coin weighted $\theta$ producing HHT is 

$$L(\theta) = P(\textrm{Data} | \theta) = \theta^2(1-\theta)$$

The maximum value of this function over $[0,1]$ can be found using differential calculus:

$$
\frac{\textrm{d} L}{\textrm{d} \theta} =2\theta - 3\theta^2
$$

So the derivative is $0$ when $\theta = 0$ or $\theta = \frac{2}{3}$.  As $L(0) = L(1) = 0$ and $L(\frac{2}{3}) > 0$ we have that the global maximum of $L$ occurs at $\theta = \frac{2}{3}$.  

Does this agree with the guess you made before doing the Calculus?

Let's do the same thing, but imagine that you tossed the coin $n$ times, got $m$ heads and $n-m$ tails.

The probability of a coin weighted $\theta$ producing $m$ heads and $n-m$ tails.

$$L(\theta) = \theta^m(1-\theta)^{n-m}$$

We could differentiate this directly, but we will get a cleaner result if we use logarithmic differentiation.

$$
\begin{align*}
\log(L)(\theta) &= \log(\theta^m(1-\theta)^{n-m})\\
&= m\log(\theta) + (n-m)\log(1-\theta)\\
\frac{\textrm{d}}{\textrm{d}\theta} \log(L(\theta)) = \frac{m}{\theta} - \frac{n-m}{1-\theta}
\end{align*}
$$

Setting this equal to zero we have

$$
\begin{align*}
\frac{m}{\theta} - \frac{n-m}{1-\theta} &= 0\\
m(1-\theta) - (n-m)\theta &= 0\\
m - m\theta - n\theta + m\theta &= 0\\
\theta &= \frac{m}{n} 
\end{align*}
$$

which should also agree with your intuition!  

#### MAP

Real world coins have $\theta \approx \frac{1}{2}$.  In the situation above we are **told** that the coin (might be) biased, and so it is reasonable to guess that $\theta = \frac{4}{5}$ if you observe $4$ heads and $1$ tail.  Without being given any such information, we might instead think that the distribution of weights could look something like this:

<p align="center">

<img src="math_hour_assets/beta_prior.png" alt="beta prior" style="width:200px;"/>

</p>

This is an example of a "prior distribution" on the parameter $\theta$.  This particular prior is 

$$
P(\theta) = k \theta^8 (1-\theta)^8
$$

where the constant $k$ has been chosen to make the integral equal to $1$ over $[0,1]$ (it turns out that $k = \frac{1}{B(9,9)}$ where $B$ is the "beta" function. See [beta distribution](https://en.wikipedia.org/wiki/Beta_distribution) for more info).

Assume that we flip our coin $5$ times and obtain $4$ heads as before. Instead of estimating $\theta = \frac{m}{n}$ we should now guess something in between $\frac{4}{5}$ and $\frac{1}{2}$.  How can we calculate it exactly?  Bayes' Rule to the rescue!

$$
\begin{align*}
P(\theta | \textrm{Data}) 
&\propto P(\textrm{Data}| \theta) \cdot P(\theta)\\
&\propto \theta^4(1-\theta)^1 \cdot \theta^8(1-\theta)^8\\
&\propto \theta^{4+8}(1-\theta)^{1+8}\\
&= \theta^{12} (1 - \theta)^9
\end{align*}
$$

We have maximized this kind of thing before:  the maximum value occurs at $\hat{\theta} = \frac{12}{21}$.

Note that $P(\theta | \textrm{Data}) $ is the "posterior probability".  We are maximizing this term, hence "maximum a posteriori" estimation.  To summarize the terminology:

$$
\begin{align*}
P(\theta | \textrm{Data}) 
&\propto P(\textrm{Data}| \theta) \cdot P(\theta)\\
\textrm{Posterior} &\propto \textrm{Likelihood} \cdot \textrm{Prior}
\end{align*}
$$

We say that our prior was equivalent to $16$ ["psuedo-observations"](https://en.wikipedia.org/wiki/Conjugate_prior#Pseudo-observations) with $8$ heads and $8$ tails.  This is a really nice way of giving some intuition to what our prior was:  intuitively you could say something like "Since most coins I have seen are balanced, I will give this coin a 'head start' of 8 heads and 8 tails before estimating my parameter $\theta$".

In general MLE is equivalent to MAP with a uniform prior (in terms of psuedo-observations:  giving no head start at all).  

### Using NumPy to generate synthetic coin flips.

In [ ]:
import numpy as np

rng = np.random.default_rng(216)  # Representing my home town till the day I die!

draws = rng.binomial(
    n=1, p=0.6, size=1000
)  # 1000 random coin flips with heads weighted to come up 60% of the time.
p_estimate = draws.sum() / 1000
print("Coin flips \n", draws)
print("Estimate of the weight =", p_estimate)

## MLE estimates of mean and variance

We now leave the discrete world.

Assume that $n$ observations $x_1, x_2, x_3, \dots , x_n$ are drawn from a normal distribution with (unknown) mean $\mu$ and (unknown) variance $\sigma^2$.

We want to find the maximum likelihood estimates $\hat{\mu}$ and $\hat{\sigma}^2$ of these parameters.  You might be able to guess what these are!  Absent any other information, what would your best guess for $\mu$ and $\sigma$ be?

We want to maximize

$$
\operatorname{L}(\mu, \sigma) = \prod_{i=1}^{n} \frac{1}{\sqrt{2\pi} \sigma} \exp\left(-\frac{(x_i-\mu)^2}{2\sigma^2}\right)
$$

Again, it is more convenient to turn this product into a sum using the logarithm. This is the **log likelihood**.

$$
\operatorname{LL}(\mu, \sigma) = \sum_{i=1}^{n} \log\left( \frac{1}{\sqrt{2\pi} \sigma} \exp\left(-\frac{(x_i-\mu)^2}{2\sigma^2}\right) \right)
$$

After a little algebra we have 

$$
\operatorname{LL}(\mu, \sigma)  = \left[ -n \log{\sqrt{2\pi}} - n\log(\sigma) - \sum_{i=1}^{n} \frac{(x_i-\mu)^2}{2\sigma^2} \right]
$$

This is a lot of minus signs, so we can instead think about minimizing the **negative log likelihood**

$$
\operatorname{NLL}(\mu, \sigma) = \left[ n \log{\sqrt{2\pi}} + n\log(\sigma) + \sum_{i=1}^{n} \frac{(x_i-\mu)^2}{2\sigma^2}\right]
$$

I will switch to using $\ell$ in place of $\operatorname{NLL}$ to make it look pretty.

We can now take partial derivatives

$$
\begin{align*}
\frac{\partial \ell}{\partial \mu} &= -\sum_{i=1}^{n} \frac{(x_i-\mu)}{\sigma^2}\\
\frac{\partial \ell}{\partial \sigma} &= \frac{n}{\sigma} - \sum_{i=1}^{n} \frac{(x_i-\mu)^2}{\sigma^3}
\end{align*}
$$

Setting both equal to $0$ and solving we obtain

$$
\begin{align*}
\hat{\mu} &= \frac{1}{n} \sum_{i=1}^{n} x_i = \bar{x}\\
\hat{\sigma}^2 &= \frac{1}{n} \sum_{i=1}^{n} (x_i - \bar{x})^2
\end{align*}
$$


In [ ]:
# Generating synthetic data from a normal distribution

mean = 5
sigma = 3  # standard deviation is 3, so variance is 9.
num_draws = 10000

rng = np.random.default_rng(216)
draws = rng.normal(
    loc=mean, scale=sigma, size=num_draws
)  # 10000 draws from a normal with mean 5 and standard deviation 3.

sample_mean = (1 / num_draws) * draws.sum()
sample_variance = pow((draws - sample_mean), 2).sum() / num_draws
print(sample_mean)
print(sample_variance)

### Bessel's correction

You may have learned the formula 

$$
\hat{\sigma}^2 = \frac{1}{n-1} \sum_{i=1}^{n} (x_i - \bar{x})^2
$$

which differs from the MLE estimate by a factor of $\frac{n}{n-1}$.  What is up with that?

I will temporarily use $\hat{\sigma}^2_{\textrm{mle}}$ and $\hat{\sigma}^2_{\textrm{bessel}}$ to distinguish between the two.

The sample mean $\bar{x}$ is not exactly equal to the true population mean $\mu$.  Let's see how that difference contributes to the MLE estimate:

Since 

$$
\begin{align*}
(x_i - \bar{x})^2 
&= (x_i - \mu + \mu - \bar{x})^2\\
&= (x_i - \mu)^2 + (\mu - \bar{x})^2 + 2(x_i - \mu)(\mu - \bar{x})
\end{align*}
$$

Summing over all samples we have

$$
\begin{align*}
 \sum_{i=1}^{n} (x_i - \bar{x})^2 
&= n(\mu - \bar{x})^2 + \sum_{i=1}^{n} (x_i - \mu)^2 + 2(\mu - \bar{x})\sum_{i=1}^{n} (x_i - \mu)
\end{align*}
$$

Observe that $\sum_{i=1}^{n} (x_i - \mu) = n(\bar{x} - \mu)$, so we have

$$
 \sum_{i=1}^{n} (x_i - \bar{x})^2  = \left(\sum_{i=1}^{n} (x_i - \mu)^2\right) - n(\mu - \bar{x})^2
$$

We thus have that

$$
\hat{\sigma}^2_{\textrm{mle}} = \frac{1}{n}\sum_{i=1}^{n} (x_i - \bar{x})^2  = \frac{1}{n}\left(\sum_{i=1}^{n} (x_i - \mu)^2\right) - (\mu - \bar{x})^2
$$

In other words, our MLE estimate of the variance $\hat{\sigma}^2_{\textrm{mle}}$ is always going to be a bit **smaller** than the true variance.  The factor of $\frac{n}{n-1}>1$ serves to "pump it up" a little to compensate.  Why do we choose exactly this factor?

So far we have been working with one particular sample $x_1, x_2, x_3, ..., x_n$.  In discussing the properties of the MLE and Bessel corrected variance estimators we want to think about the expected value of these estimators thought of as random variables themselves.  So we will let $X_1, X_2, ..., X_n$ be identically distributed independent normal random variables with mean $\mu$ and variance $\sigma^2$.

Let's calculate the expected value of $\hat{\sigma}^2_{\textrm{mle}}$.

$$
\begin{align*}
\mathbb{E}(\hat{\sigma}^2_{\textrm{mle}}) = \mathbb{E}\left(\frac{1}{n}\sum_{i=1}^{n} (X_i - \mu)^2\right) - \mathbb{E}\left((\mu - \bar{X})^2\right)
\end{align*}
$$

First note that the expected value of $(X - \mu)^2$ for any $X \sim N(\mu, \sigma^2)$ is $\sigma^2$ as shown [here](https://math.stackexchange.com/a/519631/34287).  So 

$$\mathbb{E}\left(\frac{1}{n}\sum_{i=1}^{n} (X_i - \mu)^2\right) = \sigma^2$$

 by linearity of expectation. 

Now we compute the second term:

$$
\begin{align*}
\mathbb{E}\left((\mu - \bar{X})^2\right)
&= \mathbb{E}\left((\mu - \frac{1}{n} \sum_1^n X_i)^2\right)\\
&= \mathbb{E}\left((\frac{1}{n} \sum_1^n (X_i - \mu))^2 + \sum_{i \neq j} (X_i - \mu)(X_j - \mu)\right)\\
&= \frac{1}{n}\sigma^2 + \mathbb{E}\left(\sum_{i \neq j} (X_i - \mu)(X_j - \mu) \right)
\end{align*}
$$

Note that 

$$
\begin{align*}
\mathbb{E}\left(\sum_{i \neq j} (X_i - \mu)(X_j - \mu) \right)
&= \sum_{i \neq j}  \mathbb{E}\left( (X_i - \mu)(X_j - \mu)\right) \textrm{ by linearity of expectation}\\
&= \sum_{i \neq j}  \mathbb{E}(X_i - \mu)\mathbb{E}(X_j - \mu) \textrm{ since $X_i, X_j$ are independent}\\
&= 0
\end{align*}
$$


Thus we have

$$
\mathbb{E}(\hat{\sigma}^2_{\textrm{mle}}) = \sigma^2 - \frac{1}{n}\sigma^2 = \frac{n-1}{n} \sigma^2
$$

which we multiply by $\frac{n}{n-1}$ to yield Bessel's correction.

$$
\mathbb{E}(\hat{\sigma}^2_{\textrm{bessel}}) = \sigma^2
$$

Both estimates $\hat{\sigma}^2_{\textrm{mle}}$ and $\hat{\sigma}^2_{\textrm{bessel}}$ converge ([in probability](https://en.wikipedia.org/wiki/Convergence_of_random_variables#Convergence_in_probability)) to the true value $\sigma^2$ as the number of samples grows without bounds.  In other words both estimators are [**consistent**](https://en.wikipedia.org/wiki/Consistent_estimator).  However, $\hat{\sigma}^2_{\textrm{mle}}$ is a [**biased**](https://en.wikipedia.org/wiki/Bias_of_an_estimator) estimator while **$\hat{\sigma}^2_{\textrm{bessel}}$** is an **unbiased** estimator.  The difference disappears with a large number of samples.  Is one preferable to the other?  It is really a matter of taste.


Let's inspect the performance difference using synthetic data.  We will calculate both estimates 10000 times, using only 4 samples for each draw.

In [ ]:
# Seeing Bessel's correction with examples:

rng = np.random.default_rng(216)

mean = 5
sigma = 3  # standard deviation is 3, so variance is 9.
num_draws = 4
num_samples = 10000

# Generate a (10000,4) matrix of standard normal draws
X = rng.normal(loc=mean, scale=sigma, size=(10000, 4))

# Compute the sample mean for each row
X_mean = np.mean(X, axis=1)  # (10000,)

# Compute the sum of squared deviations from the mean
sum_sq_dev = np.sum((X - X_mean.reshape(-1, 1)) ** 2, axis=1)  # (10000,)

# Compute variance estimates
var_mle = sum_sq_dev / num_draws  # MLE estimate (divide by n)
var_bessel = sum_sq_dev / (num_draws - 1)  # Bessel's corrected estimate (divide by n-1)

# Compute the mean of each variance estimate
mean_var_mle = np.mean(var_mle)
mean_var_bessel = np.mean(var_bessel)

print(f"The mean MLE estimate is this far off: {sigma**2 - mean_var_mle}")
print(f"The mean Bessel estimate is this far off: {sigma**2 - mean_var_bessel}")

We can see that the Bessel corrected estimate does a much better job, on average, of estimating the variance when we have a small number of samples.  

Despite the poor performance of MLE on few samples, it is almost as good as the Bessel's correct estimator with a large number of samples.

We will use MLE estimators frequently in this course without attempting to make them unbiased.  They are consistent estimators, and in data science we will use large amounts of data.  So it is easier to just use MLE!

## Where to go from here?

We will use MLE constantly in math hour!  In particular, MLE comes up in:

* Linear Regression
* Logistic Regression
* Gaussian Discriminant Analysis

MAP will return as one way to understand regularization.

More generally, MLE and MAP follow a common machine learning pattern which we will see again and again:

* We have some data.
* We create a model for a data generating process which depends on some parameters.
* We specify a "loss function": a quantitative measure of "how well" a given set of parameters models the data.
    * In the case of MLE and MAP our loss function is negative log Likelihood.
    * Another common loss function is mean squared error.
    * We also sometimes modify our loss function through regularization such as adding a penalty term of the form $\lambda |\theta|^2$ to the loss.
* We minimize the loss function, often by using gradient descent when exact solutions are unavailable.